In [22]:
import json
from pathlib import Path

import pandas as pd

In [23]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-sweep-semi/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[1]

601 experiments


{'commit': 'c9ffff5dd2706e29c13cbc66cd0551cfb9dcdd1b',
 'id': '5bf0f42a3be03329be9d148689ff94f29ae6eebe',
 'name': 'irate-cons',
 'params': {'dataset': {'path': 'bdsaglam/musique-sweep',
   'name': 'answerable',
   'split': 'train'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.1,
   'system_prompt': 'cot/original.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'semi/cot-original.json',
   'n_shot': 0},
  'run': 2},
 'metrics': {'exact_match': 0.6033333333333334,
  'f1': 0.7033493455861877,
  'fuzzy_match': 0.6766666666666666,
  '2hops': {'exact_match': 0.71,
   'f1': 0.8188099415204678,
   'fuzzy_match': 0.81},
  '3hops': {'exact_match': 0.58,
   'f1': 0.6703333333333333,
   'fuzzy_match': 0.64},
  '4hops': {'exact_match': 0.52,
   'f1': 0.6209047619047618,
   'fuzzy_match': 0.58}}}

In [24]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-sweep")  &
    (df["params.qa.few_shot_examples"].str.contains("semi")) &
    (df["metrics.f1"] > 0.0)
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

df['params.qa.n_sc'] = df['params.qa.n_sc'].fillna(1)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols + metric_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

402 experiments after preprocessing


In [25]:
def parse_qa_technique(row):
    base_technique = row["params.qa.system_prompt"].split("/")[0].upper()
    sc = row["params.qa.n_sc"]
    sc_suffix = f"-SC{sc}" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [26]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.n_sc",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.n_sc,params.qa.temperature,params.run
3,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,10.0,0.5,"(1, 3)"
4,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,10.0,0.7,"(1, 3)"
8,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,10.0,0.1,"(1,)"
9,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,10.0,0.5,"(1,)"
10,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,10.0,0.7,"(1,)"
...,...,...,...,...,...,...,...
162,direct/format-few.txt,cq.txt,semi/direct.json,2,10.0,0.5,"(1,)"
163,direct/format-few.txt,cq.txt,semi/direct.json,2,10.0,0.7,"(1,)"
167,direct/format-few.txt,cq.txt,semi/direct.json,3,10.0,0.1,"(1, 3)"
168,direct/format-few.txt,cq.txt,semi/direct.json,3,10.0,0.5,"(2,)"


In [27]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/format-thought.txt
	ccot/original.txt
	cok/adapted.txt
	cok/original.txt
	cot/format-thought.txt
	cot/original.txt
	cte/format-triplets-few.txt
	direct/format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	semi/ccot.json
	semi/ccot-original.json
	semi/cok.json
	semi/cot.json
	semi/cot-original.json
	semi/cte.json
	semi/direct.json
- params.qa.n_shot
	0
	1
	2
	3
- params.qa.n_sc
	1.0
	10.0
- params.qa.temperature
	0.1
	0.5
	0.7
- params.run
	(1, 2, 3)
	(1, 3)
	(1,)
	(2,)
	(3,)
	(2, 3)
	(1, 2)


In [28]:
df.to_json('exps.jsonl', orient='records', lines=True)